In [141]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import keras
import numpy as np
import pandas as pd
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten, BatchNormalization,concatenate,Input
from keras.layers.core import Permute, Reshape
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import Regularizer
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.utils import to_categorical

%matplotlib inline  

### Opening Face Points

In [112]:
FILENAME = 'features_points.h5'
h5f = h5py.File(FILENAME,'r')

for key in h5f.keys():
    print(key) #Names of the groups in HDF5 file.
x = h5f['points'][:]

points


### Getting AUX input into system

In [113]:
aux_inputs = pd.read_csv('aux_inputs.csv')

In [114]:
aux_inputs.head()

,SessionID,Female (y=1),Length(s)
0,S001-001-l000,0,0.822
1,S001-001-l001,0,0.912
2,S001-001-l002,0,0.834
3,S001-003-l000,0,0.552
4,S001-003-l001,0,0.576


### Getting Common Train/Test Splits

In [115]:
import  pickle

with open("train_indx.p", 'rb') as pickleFile:
    train_idx = pickle.load(pickleFile)
with open("test_indx.p", 'rb') as pickleFile:
    test_idx = pickle.load(pickleFile)

In [116]:
print(len(train_idx))
print(len(test_idx))

405
209


### Getting targets in System

In [165]:
targets = pd.read_csv('merged_labels.csv')

target_train = targets.iloc[train_idx]
target_test = targets.iloc[test_idx]

target_train.head()

,SessionID,Session,Type,Merged Arousal,Merged Dominance,Merged Emotion
406,S019-001-l005,130,Laughter,4,4,Schadenfreude
32,S002-005-l005,13,Laughter,3,2,Happy
196,S007-007-l003,62,Laughter,2,2,Happy
265,S011-001-l002,76,Laughter,2,3,Schadenfreude
8,S001-003-l005,3,Laughter,3,2,Schadenfreude


In [166]:
y_train = np.array(target_train['Merged Arousal'])
y_test = np.array(target_test['Merged Arousal'])
y1_train = target_train['Merged Dominance']
y1_test = target_test['Merged Dominance']
y2_train = target_train['Merged Emotion']
y2_test = target_test['Merged Emotion']

aux_inputs_train = np.array(aux_inputs[['Female (y=1)','Length(s)']].iloc[train_idx])
aux_inputs_test = np.array(aux_inputs[['Female (y=1)','Length(s)']].iloc[test_idx])

x_train = x[train_idx,:,:,:]
x_test = x[test_idx,:,:,:]

x_train = x_train.reshape((len(train_idx),25,-1))
x_test = x_test.reshape((len(test_idx),25,-1))

x_train = np.swapaxes(x_train,1,2)
x_train = np.reshape(x_train, (-1, feat_dim, WINDOW_SIZE, 1))
x_test = np.swapaxes(x_test,1,2)
x_test = np.reshape(x_test, (-1, feat_dim, WINDOW_SIZE, 1))

x_train.shape

(405, 136, 25, 1)

In [119]:
def create_graph_arousal_dominance():
    
    aux_input = Input(shape=(2,),dtype='float32', name='aux_input')
    
    inputs = Input(shape=(feat_dim,WINDOW_SIZE,1), name='main_input')
    
    x = BatchNormalization()(inputs)
    
    x = Conv2D(num_feat_map, kernel_size=(1, 5),
                 activation='relu',
                 padding='same')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Conv2D(num_feat_map, kernel_size=(1, 5), activation='relu',padding='same')(x)
    x = Dropout(0.5)(x)
    
    x = Permute((2, 1, 3))(x) # for swap-dimension
    x = Reshape((-1,num_feat_map*feat_dim))(x)

    x = LSTM(num_feat_map, return_sequences=False)(x)
    x = Dropout(0.5)(x)
    x = concatenate([x,aux_input])
    main_output = Dense(1,activation='linear')(x)
    
    model = Model(inputs=[inputs, aux_input], outputs=main_output)
    
    model.summary()
    
    return model
    

# Conv LSTM Model Arousal

In [120]:
WINDOW_SIZE = 25
num_feat_map = 32
feat_dim = 136

In [121]:
model = create_graph_arousal_dominance()

(405, 136, 25)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 136, 25, 1)    0                                            
____________________________________________________________________________________________________
batch_normalization_9 (BatchNorm (None, 136, 25, 1)    4           main_input[0][0]                 
____________________________________________________________________________________________________
conv2d_9 (Conv2D)                (None, 136, 25, 32)   192         batch_normalization_9[0][0]      
____________________________________________________________________________________________________
dropout_13 (Dropout)             (None, 136, 25, 32)   0           conv2d_9[0][0]                   
____________________________________________________________________________

In [122]:
epochs = 150
batch_size=16

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
#filepath="weights_best_arousal.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model.compile(loss=keras.losses.mean_squared_error,
              optimizer='adam',
              metrics=['mse'])

model.fit([x_train,aux_inputs_train], y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_data=([x_test,aux_inputs_test], y_test))

Train on 405 samples, validate on 209 samples
Epoch 1/150
405/405 [==============================] - 8s - loss: 1.7127 - mean_squared_error: 1.7127 - val_loss: 1.0030 - val_mean_squared_error: 1.0030
Epoch 2/150
405/405 [==============================] - 2s - loss: 1.3646 - mean_squared_error: 1.3646 - val_loss: 1.0236 - val_mean_squared_error: 1.0236
Epoch 3/150
405/405 [==============================] - 2s - loss: 1.4056 - mean_squared_error: 1.4056 - val_loss: 0.9978 - val_mean_squared_error: 0.9978
Epoch 4/150
405/405 [==============================] - 2s - loss: 1.3731 - mean_squared_error: 1.3731 - val_loss: 1.0188 - val_mean_squared_error: 1.0188
Epoch 5/150
405/405 [==============================] - 2s - loss: 1.3698 - mean_squared_error: 1.3698 - val_loss: 1.0621 - val_mean_squared_error: 1.0621
Epoch 6/150
405/405 [==============================] - 2s - loss: 1.3605 - mean_squared_error: 1.3605 - val_loss: 1.0027 - val_mean_squared_error: 1.0027
Epoch 7/150
405/405 [=========

405/405 [==============================] - 2s - loss: 0.9362 - mean_squared_error: 0.9362 - val_loss: 0.6893 - val_mean_squared_error: 0.6893
Epoch 54/150
405/405 [==============================] - 2s - loss: 0.9733 - mean_squared_error: 0.9733 - val_loss: 0.7744 - val_mean_squared_error: 0.7744
Epoch 55/150
405/405 [==============================] - 2s - loss: 1.0378 - mean_squared_error: 1.0378 - val_loss: 0.7129 - val_mean_squared_error: 0.7129
Epoch 56/150
405/405 [==============================] - 2s - loss: 1.0447 - mean_squared_error: 1.0447 - val_loss: 0.8068 - val_mean_squared_error: 0.8068
Epoch 57/150
405/405 [==============================] - 2s - loss: 1.0599 - mean_squared_error: 1.0599 - val_loss: 0.7592 - val_mean_squared_error: 0.7592
Epoch 58/150
405/405 [==============================] - 2s - loss: 1.1131 - mean_squared_error: 1.1131 - val_loss: 0.7146 - val_mean_squared_error: 0.7146
Epoch 59/150
405/405 [==============================] - 2s - loss: 1.0130 - mean_sq

405/405 [==============================] - 2s - loss: 0.9086 - mean_squared_error: 0.9086 - val_loss: 0.8197 - val_mean_squared_error: 0.8197
Epoch 106/150
405/405 [==============================] - 2s - loss: 1.0230 - mean_squared_error: 1.0230 - val_loss: 0.8171 - val_mean_squared_error: 0.8171
Epoch 107/150
405/405 [==============================] - 2s - loss: 1.0555 - mean_squared_error: 1.0555 - val_loss: 0.8026 - val_mean_squared_error: 0.8026
Epoch 108/150
405/405 [==============================] - 2s - loss: 0.9961 - mean_squared_error: 0.9961 - val_loss: 0.7276 - val_mean_squared_error: 0.7276
Epoch 109/150
405/405 [==============================] - 2s - loss: 0.9121 - mean_squared_error: 0.9121 - val_loss: 0.6964 - val_mean_squared_error: 0.6964
Epoch 110/150
405/405 [==============================] - 2s - loss: 0.9961 - mean_squared_error: 0.9961 - val_loss: 0.7184 - val_mean_squared_error: 0.7184
Epoch 111/150
405/405 [==============================] - 2s - loss: 0.9772 - m

In [124]:
preds = model.predict([x_test,aux_inputs_test])
preds = preds.reshape(preds.shape[0])

#### Prediction Eval

In [125]:
mean_y = np.mean(preds)
print("Mean of predictions is: "+str(mean_y))

mse = np.mean((y_test - preds)**2)
print("Variance (benchmark) is: "+str(mse))

Mean of predictions is: 1.88531
Variance (benchmark) is: 0.701500721508


In [126]:
preds

array([ 2.17263126,  1.04617345,  1.64479566,  1.23388839,  2.86100817,
        3.71547222,  2.80520201,  3.64250803,  3.14486122,  2.18505716,
        2.1777451 ,  0.96880531,  2.20417738,  0.96121305,  1.0687139 ,
        1.14163768,  0.99956346,  0.95929384,  0.97007936,  1.22787547,
        1.15225303,  1.22155011,  1.13291335,  1.02378726,  1.53078473,
        1.90811276,  1.2129823 ,  2.43585014,  0.99306613,  0.98058033,
        1.00851059,  2.90824413,  2.03819227,  1.45416629,  1.81926012,
        1.81660342,  1.612082  ,  1.82738423,  1.70068717,  1.47721899,
        2.08559966,  1.57050943,  1.21196914,  1.3934505 ,  2.43544984,
        2.39549494,  1.94811702,  1.42889047,  1.57413816,  1.11355615,
        1.34874034,  1.05631065,  1.59055412,  1.06175244,  1.05425107,
        1.40130591,  1.06867611,  1.80071044,  1.94403291,  2.43722987,
        0.98168635,  1.04482853,  0.97133076,  2.22818112,  0.97025025,
        0.99537861,  0.98309267,  1.02088857,  1.07566464,  0.96

#### Benchmark Eval

In [127]:
mean_y = np.mean(y_test)
print("Mean of test set is: "+str(mean_y))

variance = np.mean((y_test - np.mean(y_test))**2)
print("Variance (benchmark) is: "+str(variance))

Mean of test set is: 1.86124401914
Variance (benchmark) is: 1.28696687347


# Conv LSTM Dominance

In [136]:
WINDOW_SIZE = 25
num_feat_map = 32
feat_dim = 136

#x_train, x_test, y_train, y_test = train_test_split(x, y1, test_size=0.2, random_state=42)

#x_train = x_train.reshape((491,25,-1))
#x_test = x_test.reshape((123,25,-1))

#x_train = np.swapaxes(x_train,1,2)
#x_train = np.reshape(x_train, (-1, feat_dim, WINDOW_SIZE, 1))
#x_test = np.swapaxes(x_test,1,2)
#x_test = np.reshape(x_test, (-1, feat_dim, WINDOW_SIZE, 1))

model = create_graph_arousal_dominance()

epochs = 50
batch_size=16

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
sgd = keras.optimizers.SGD(lr=0.01, clipnorm=1.)
filepath="weights_best_arousal.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#adam1 = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss=keras.losses.mean_squared_error,
              optimizer='adam',
              metrics=['mse'])

model.fit([x_train,aux_inputs_train], y1_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_data=([x_test,aux_inputs_test], y1_test))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 136, 25, 1)    0                                            
____________________________________________________________________________________________________
batch_normalization_17 (BatchNor (None, 136, 25, 1)    4           main_input[0][0]                 
____________________________________________________________________________________________________
conv2d_17 (Conv2D)               (None, 136, 25, 32)   192         batch_normalization_17[0][0]     
____________________________________________________________________________________________________
dropout_25 (Dropout)             (None, 136, 25, 32)   0           conv2d_17[0][0]                  
___________________________________________________________________________________________

405/405 [==============================] - 2s - loss: 0.8091 - mean_squared_error: 0.8091 - val_loss: 0.6256 - val_mean_squared_error: 0.6256
Epoch 32/50
405/405 [==============================] - 2s - loss: 0.8096 - mean_squared_error: 0.8096 - val_loss: 0.6222 - val_mean_squared_error: 0.6222
Epoch 33/50
405/405 [==============================] - 2s - loss: 0.8315 - mean_squared_error: 0.8315 - val_loss: 0.6322 - val_mean_squared_error: 0.6322
Epoch 34/50
405/405 [==============================] - 2s - loss: 0.9056 - mean_squared_error: 0.9056 - val_loss: 0.6251 - val_mean_squared_error: 0.6251
Epoch 35/50
405/405 [==============================] - 2s - loss: 0.7768 - mean_squared_error: 0.7768 - val_loss: 0.6572 - val_mean_squared_error: 0.6572
Epoch 36/50
405/405 [==============================] - 2s - loss: 0.8196 - mean_squared_error: 0.8196 - val_loss: 0.6478 - val_mean_squared_error: 0.6478
Epoch 37/50
405/405 [==============================] - 2s - loss: 0.8469 - mean_squared_

#### Eval

In [137]:
preds = model.predict([x_test,aux_inputs_test])
preds = preds.reshape(preds.shape[0])

mean_y = np.mean(preds)
print("Mean of predictions is: "+str(mean_y))

mse = np.mean((y1_test - preds)**2)
print("Variance (benchmark) is: "+str(mse))

print()

mean_y = np.mean(y1_test)
print("Mean of test set is: "+str(mean_y))

variance = np.mean((y1_test - np.mean(y_test))**2)
print("Variance (benchmark) is: "+str(variance))

Mean of predictions is: 2.43652
Variance (benchmark) is: 0.6222497387116338

Mean of test set is: 2.311004784688995
Variance (benchmark) is: 0.9046038323298448


In [138]:
preds

array([ 2.4474628 ,  2.27879333,  2.20608926,  1.71106207,  2.82110453,
        2.98488808,  2.26071262,  2.65786552,  2.75414515,  2.37763596,
        2.39434695,  2.04531622,  2.4380939 ,  2.2634933 ,  2.43008089,
        2.14567041,  2.44361067,  2.41811013,  2.39364195,  2.28936744,
        2.3412509 ,  2.22638297,  2.39908981,  2.40327334,  2.35173202,
        2.59816599,  2.44988489,  2.79702067,  2.14157867,  2.10602689,
        2.13534474,  2.99174976,  2.3427465 ,  2.20174956,  2.47160602,
        2.31461287,  2.30588388,  2.66174126,  2.35055614,  2.26085448,
        2.30857873,  2.34453249,  2.20147896,  2.23436904,  2.65773845,
        2.59819198,  2.50496197,  2.28457284,  2.32651067,  2.21581936,
        2.06962872,  1.95111072,  2.2014792 ,  2.06576657,  2.00407839,
        2.23171091,  2.08660746,  2.32423878,  2.26668525,  2.32133341,
        1.90671766,  2.13174272,  1.81924582,  2.53938198,  2.03945065,
        2.01759434,  2.06604242,  2.04603148,  2.37366796,  2.08

# Conv LSTM Model Emotions

In [179]:
def create_graph_emotions():
    
    aux_input = Input(shape=(2,),dtype='float32', name='aux_input')
    
    inputs = Input(shape=(feat_dim,WINDOW_SIZE,1), name='main_input')
    
    x = BatchNormalization()(inputs)
    
    x = Conv2D(num_feat_map, kernel_size=(1, 5),
                 activation='relu',
                 padding='same')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Conv2D(num_feat_map, kernel_size=(1, 5), activation='relu',padding='same')(x)
    x = Dropout(0.5)(x)
    
    x = Permute((2, 1, 3))(x) # for swap-dimension
    x = Reshape((-1,num_feat_map*feat_dim))(x)

    x = LSTM(num_feat_map, return_sequences=False)(x)
    x = Dropout(0.5)(x)
    x = concatenate([x,aux_input])
    x = BatchNormalization()(x)
    
    main_output = Dense(4,activation='softmax')(x)
    
    model = Model(inputs=[inputs, aux_input], outputs=main_output)
    
    model.summary()
    
    return model

In [180]:
y2_train2.shape

(405, 4)

In [191]:
WINDOW_SIZE = 25
num_feat_map = 32
feat_dim = 136

model = Sequential()

label_encoder = LabelEncoder()
one_hot = OneHotEncoder(sparse=False)
y2_train1 = label_encoder.fit_transform(y2_train)
y2_train2 = one_hot.fit_transform(y2_train1.reshape(len(y2_train1),1))
y2_test1 = label_encoder.fit_transform(y2_test)
y2_test2 = one_hot.fit_transform(y2_test1.reshape(len(y2_test1),1))

model = create_graph_emotions()

epochs = 400
batch_size=32

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
#filepath="weights_best_arousal.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='sgd',
              metrics=['accuracy'])

model.fit([x_train,aux_inputs_train], y2_train2,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_data=([x_test,aux_inputs_test], y2_test2))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 136, 25, 1)    0                                            
____________________________________________________________________________________________________
batch_normalization_44 (BatchNor (None, 136, 25, 1)    4           main_input[0][0]                 
____________________________________________________________________________________________________
conv2d_39 (Conv2D)               (None, 136, 25, 32)   192         batch_normalization_44[0][0]     
____________________________________________________________________________________________________
dropout_58 (Dropout)             (None, 136, 25, 32)   0           conv2d_39[0][0]                  
___________________________________________________________________________________________

405/405 [==============================] - 1s - loss: 1.2564 - acc: 0.4444 - val_loss: 1.2360 - val_acc: 0.4593
Epoch 38/400
405/405 [==============================] - 1s - loss: 1.2676 - acc: 0.4593 - val_loss: 1.2311 - val_acc: 0.4593
Epoch 39/400
405/405 [==============================] - 1s - loss: 1.2481 - acc: 0.4173 - val_loss: 1.2511 - val_acc: 0.4593
Epoch 40/400
405/405 [==============================] - 2s - loss: 1.2694 - acc: 0.4247 - val_loss: 1.2480 - val_acc: 0.4593
Epoch 41/400
405/405 [==============================] - 2s - loss: 1.2393 - acc: 0.4543 - val_loss: 1.2095 - val_acc: 0.4641
Epoch 42/400
405/405 [==============================] - 1s - loss: 1.2494 - acc: 0.4691 - val_loss: 1.2067 - val_acc: 0.4593
Epoch 43/400
405/405 [==============================] - 2s - loss: 1.2561 - acc: 0.4444 - val_loss: 1.2029 - val_acc: 0.4785
Epoch 44/400
405/405 [==============================] - 1s - loss: 1.2530 - acc: 0.4593 - val_loss: 1.1970 - val_acc: 0.4833
Epoch 45/400


405/405 [==============================] - 1s - loss: 1.1951 - acc: 0.4963 - val_loss: 1.1820 - val_acc: 0.4928
Epoch 103/400
405/405 [==============================] - 1s - loss: 1.2045 - acc: 0.4889 - val_loss: 1.1914 - val_acc: 0.4928
Epoch 104/400
405/405 [==============================] - 1s - loss: 1.1879 - acc: 0.4938 - val_loss: 1.1956 - val_acc: 0.4880
Epoch 105/400
405/405 [==============================] - 1s - loss: 1.1906 - acc: 0.4988 - val_loss: 1.1911 - val_acc: 0.4928
Epoch 106/400
405/405 [==============================] - 1s - loss: 1.1871 - acc: 0.4716 - val_loss: 1.1900 - val_acc: 0.4785
Epoch 107/400
405/405 [==============================] - 1s - loss: 1.1773 - acc: 0.5012 - val_loss: 1.1766 - val_acc: 0.4833
Epoch 108/400
405/405 [==============================] - 1s - loss: 1.1823 - acc: 0.4765 - val_loss: 1.1993 - val_acc: 0.4928
Epoch 109/400
405/405 [==============================] - 1s - loss: 1.1731 - acc: 0.5111 - val_loss: 1.1896 - val_acc: 0.4976
Epoch 

405/405 [==============================] - 1s - loss: 1.1628 - acc: 0.5185 - val_loss: 1.1773 - val_acc: 0.4928
Epoch 168/400
405/405 [==============================] - 1s - loss: 1.1659 - acc: 0.5037 - val_loss: 1.1777 - val_acc: 0.4880
Epoch 169/400
405/405 [==============================] - 1s - loss: 1.1644 - acc: 0.4938 - val_loss: 1.1679 - val_acc: 0.4976
Epoch 170/400
405/405 [==============================] - 1s - loss: 1.1833 - acc: 0.5062 - val_loss: 1.1706 - val_acc: 0.5072
Epoch 171/400
405/405 [==============================] - 1s - loss: 1.1532 - acc: 0.5284 - val_loss: 1.1605 - val_acc: 0.4976
Epoch 172/400
405/405 [==============================] - 1s - loss: 1.1631 - acc: 0.5185 - val_loss: 1.1671 - val_acc: 0.5072
Epoch 173/400
405/405 [==============================] - 1s - loss: 1.1666 - acc: 0.5210 - val_loss: 1.1679 - val_acc: 0.4928
Epoch 174/400
405/405 [==============================] - 1s - loss: 1.1529 - acc: 0.5136 - val_loss: 1.1752 - val_acc: 0.5024
Epoch 

405/405 [==============================] - 1s - loss: 1.1534 - acc: 0.5160 - val_loss: 1.1949 - val_acc: 0.4880
Epoch 233/400
405/405 [==============================] - 1s - loss: 1.1316 - acc: 0.5407 - val_loss: 1.2118 - val_acc: 0.4833
Epoch 234/400
405/405 [==============================] - 1s - loss: 1.1427 - acc: 0.5185 - val_loss: 1.2098 - val_acc: 0.5024
Epoch 235/400
405/405 [==============================] - 1s - loss: 1.1516 - acc: 0.5210 - val_loss: 1.2922 - val_acc: 0.4737
Epoch 236/400
405/405 [==============================] - 1s - loss: 1.1360 - acc: 0.5062 - val_loss: 1.2355 - val_acc: 0.4689
Epoch 237/400
405/405 [==============================] - 1s - loss: 1.1338 - acc: 0.5383 - val_loss: 1.1939 - val_acc: 0.5263
Epoch 238/400
405/405 [==============================] - 1s - loss: 1.1915 - acc: 0.4988 - val_loss: 1.1850 - val_acc: 0.5072
Epoch 239/400
405/405 [==============================] - 1s - loss: 1.1462 - acc: 0.5160 - val_loss: 1.2065 - val_acc: 0.4689
Epoch 

405/405 [==============================] - 1s - loss: 1.1226 - acc: 0.5383 - val_loss: 1.2206 - val_acc: 0.5072
Epoch 298/400
405/405 [==============================] - 1s - loss: 1.1296 - acc: 0.5432 - val_loss: 1.2618 - val_acc: 0.4928
Epoch 299/400
405/405 [==============================] - 1s - loss: 1.1044 - acc: 0.5481 - val_loss: 1.1718 - val_acc: 0.5263
Epoch 300/400
405/405 [==============================] - 1s - loss: 1.1011 - acc: 0.5481 - val_loss: 1.3100 - val_acc: 0.4833
Epoch 301/400
405/405 [==============================] - 1s - loss: 1.1153 - acc: 0.5333 - val_loss: 1.1532 - val_acc: 0.5024
Epoch 302/400
405/405 [==============================] - 1s - loss: 1.1186 - acc: 0.5358 - val_loss: 1.1757 - val_acc: 0.4928
Epoch 303/400
405/405 [==============================] - 1s - loss: 1.1124 - acc: 0.5309 - val_loss: 1.1659 - val_acc: 0.5072
Epoch 304/400
405/405 [==============================] - 1s - loss: 1.1252 - acc: 0.5284 - val_loss: 1.1772 - val_acc: 0.5167
Epoch 

405/405 [==============================] - 1s - loss: 1.1412 - acc: 0.4790 - val_loss: 1.1197 - val_acc: 0.5215
Epoch 363/400
405/405 [==============================] - 1s - loss: 1.1379 - acc: 0.5160 - val_loss: 1.1347 - val_acc: 0.5072
Epoch 364/400
405/405 [==============================] - 1s - loss: 1.1282 - acc: 0.5358 - val_loss: 1.1564 - val_acc: 0.5120
Epoch 365/400
405/405 [==============================] - 1s - loss: 1.1254 - acc: 0.5235 - val_loss: 1.1495 - val_acc: 0.5167
Epoch 366/400
405/405 [==============================] - 1s - loss: 1.1163 - acc: 0.5284 - val_loss: 1.1424 - val_acc: 0.5167
Epoch 367/400
405/405 [==============================] - 1s - loss: 1.1251 - acc: 0.5259 - val_loss: 1.1503 - val_acc: 0.5167
Epoch 368/400
405/405 [==============================] - 1s - loss: 1.1193 - acc: 0.5235 - val_loss: 1.1312 - val_acc: 0.5215
Epoch 369/400
405/405 [==============================] - 1s - loss: 1.0886 - acc: 0.5679 - val_loss: 1.1364 - val_acc: 0.5120
Epoch 

#### Eval

In [192]:
preds = model.predict([x_test,aux_inputs_test])
preds

array([[ 0.04768448,  0.0378131 ,  0.60774869,  0.30675372],
       [ 0.4703249 ,  0.08609053,  0.33651114,  0.10707346],
       [ 0.13342418,  0.06425221,  0.68945199,  0.11287159],
       [ 0.09950347,  0.08324497,  0.72535008,  0.09190145],
       [ 0.03130779,  0.05089725,  0.72246373,  0.19533123],
       [ 0.00725963,  0.08696808,  0.5577212 ,  0.34805104],
       [ 0.12774347,  0.09889749,  0.58939576,  0.18396327],
       [ 0.01198139,  0.06219753,  0.63255483,  0.29326621],
       [ 0.17877708,  0.08132731,  0.4944329 ,  0.24546267],
       [ 0.0926079 ,  0.08046555,  0.69305813,  0.13386844],
       [ 0.09533754,  0.03669759,  0.76781207,  0.10015278],
       [ 0.56097007,  0.1347709 ,  0.19282453,  0.1114345 ],
       [ 0.27894655,  0.09811429,  0.46605417,  0.15688492],
       [ 0.59160149,  0.17585067,  0.11152893,  0.12101894],
       [ 0.55355346,  0.17777236,  0.12956899,  0.13910519],
       [ 0.31736138,  0.10727215,  0.43519774,  0.14016868],
       [ 0.53038526,  0.

In [193]:
y_test3 = np.argmax(y2_test2,axis=1)
y_test3

array([3, 2, 0, 2, 2, 1, 0, 2, 2, 1, 2, 1, 1, 0, 0, 3, 1, 0, 0, 0, 1, 2, 0,
       0, 2, 2, 2, 2, 2, 0, 2, 3, 3, 0, 2, 2, 3, 3, 2, 0, 2, 3, 0, 2, 2, 3,
       2, 0, 0, 2, 1, 2, 1, 1, 1, 1, 0, 2, 2, 2, 1, 0, 1, 0, 3, 2, 3, 0, 0,
       0, 0, 1, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 3, 3, 2, 2,
       2, 2, 0, 2, 1, 0, 3, 3, 0, 2, 3, 1, 1, 0, 1, 1, 2, 0, 3, 3, 3, 3, 0,
       2, 2, 2, 0, 3, 0, 0, 3, 0, 0, 3, 0, 2, 0, 0, 3, 1, 3, 2, 3, 2, 2, 0,
       0, 1, 2, 3, 3, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 1, 3, 2, 2, 2, 3, 2,
       3, 3, 2, 0, 2, 2, 2, 2, 3, 3, 3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 0, 2,
       2, 0, 3, 2, 2, 2, 0, 3, 2, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

In [194]:
np.argmax(preds,axis=1)
confusion_matrix(y_test3,np.argmax(preds,axis=1))

array([[26,  0, 22,  1],
       [10,  0, 13,  0],
       [14,  0, 80,  2],
       [ 4,  0, 35,  2]], dtype=int64)

In [195]:
print(classification_report(y_test3,np.argmax(preds,axis=1)))

             precision    recall  f1-score   support

          0       0.48      0.53      0.50        49
          1       0.00      0.00      0.00        23
          2       0.53      0.83      0.65        96
          3       0.40      0.05      0.09        41

avg / total       0.44      0.52      0.43       209



C:\Users\Sam\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Maybe talk about resampling to take care of class imbalances in the optimization section...